In [1]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.preprocessing.image import *
from keras.utils.multi_gpu_utils import multi_gpu_model

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

from glob import glob
from tqdm import tqdm
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import *

import multiprocessing
from multiprocessing.dummy import Pool
from pprint import pprint

Using TensorFlow backend.


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'
n_gpus = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))

epochs = [40, 10, 5]
batch_size = 1 * n_gpus
model_name = 'nasnet_0801_f2'
preprocess_func = nasnet.preprocess_input
MODEL = NASNetLarge

f = 2
lr = 1e-3
width = int(2560 / f)
height = int(1920 / f)
classes = ['正常', '吊经', '擦洞', '跳花', '毛洞', '织稀', '扎洞', '缺经', '毛斑', '其他']
n_classes = len(classes)

In [3]:
print('-'*80)
print('开始训练 %s 模型。' % model_name)
print(width, height, n_classes)
print(classes)
print('-'*80)

--------------------------------------------------------------------------------
开始训练 nasnet_0801_f2 模型。
1280 960 10
['正常', '吊经', '擦洞', '跳花', '毛洞', '织稀', '扎洞', '缺经', '毛斑', '其他']
--------------------------------------------------------------------------------


In [4]:
from keras import backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

In [5]:
idg = ImageDataGenerator(horizontal_flip=True,
                         rotation_range=15,
                         zoom_range=0.2)
idg2 = ImageDataGenerator()

gen_train = idg.flow_from_directory('train', target_size=(height, width), 
                                    interpolation='lanczos', classes=classes, batch_size=batch_size)
gen_valid = idg2.flow_from_directory('valid', target_size=(height, width), shuffle=False, 
                                     interpolation='lanczos', classes=classes, batch_size=4)
y_valid = (gen_valid.classes != 0).astype(int)

Found 1816 images belonging to 10 classes.
Found 203 images belonging to 10 classes.


In [6]:
class RocAucMetricCallback(Callback):
    def __init__(self):
        super(RocAucMetricCallback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        y_pred = model_parallel.predict_generator(gen_valid, verbose=1, workers=multiprocessing.cpu_count())
        y_pred2 = 1 - y_pred[:,0]
        y_pred3 = 1 - (np.argmax(y_pred, axis=-1) == 0)
        
        auc = roc_auc_score(y_valid, y_pred2)
        acc = accuracy_score(y_valid, y_pred3)
        logs['val_auc'] = auc
        logs['val_acc'] = acc
        logs['lr'] = float(K.get_value(model_parallel.optimizer.lr))
        print(f'ACC: {acc:.4f} AUC: {auc:.4f}')

In [7]:
base_model = MODEL(input_shape=(height, width, 3), include_top=False, weights='imagenet')

In [8]:
input_tensor = Input((height, width, 3))
x = Lambda(preprocess_func)(input_tensor)
x = base_model(x)
x = GlobalAvgPool2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_classes, activation='softmax')(x)
model = Model(input_tensor, x)

In [9]:
for layer in base_model.layers + model.layers:
    if hasattr(layer, 'kernel_regularizer'):
        print(layer)
        layer.kernel_regularizer= l2(1e-5)

In [10]:
model_parallel = multi_gpu_model(model, n_gpus)
model_parallel.__setattr__('callback_model', model)

In [11]:
plot_model(model, show_shapes=True, to_file=f'logs/plot_{model_name}.png')
# SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [12]:
def train(start, end, lr, warmup=False):
    opt = SGD(lr, momentum=0.9, nesterov=True)

    callbacks = [
        RocAucMetricCallback(), 
        ModelCheckpoint('models/%s-{lr:.6f}-{epoch}-{val_auc:.6f}.h5' % model_name, 
                        save_weights_only=True), 
        CSVLogger('logs/%s.csv' % model_name, append=True), 
    ]
    
    model_parallel.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    model_parallel.fit_generator(gen_train, steps_per_epoch=len(gen_train),  
                                 workers=multiprocessing.cpu_count(), 
                                 initial_epoch=start, epochs=end, callbacks=callbacks)

In [13]:
start = 0
for i, epochs in enumerate(epochs):
    train(start, start + epochs, lr*0.1**i, warmup=i == 0)
    start += epochs

Epoch 1/40
51/51 [==============================] - 73s 1s/step
ACC: 0.6502 AUC: 0.5423
Epoch 2/40
51/51 [==============================] - 31s 608ms/step
ACC: 0.6650 AUC: 0.6641
Epoch 3/40
51/51 [==============================] - 31s 613ms/step
ACC: 0.7882 AUC: 0.8327
Epoch 4/40
51/51 [==============================] - 31s 614ms/step
ACC: 0.7833 AUC: 0.8658
Epoch 5/40
51/51 [==============================] - 31s 607ms/step
ACC: 0.7586 AUC: 0.8606
Epoch 6/40
51/51 [==============================] - 31s 611ms/step
ACC: 0.7143 AUC: 0.8622
Epoch 7/40
51/51 [==============================] - 31s 605ms/step
ACC: 0.7635 AUC: 0.8918
Epoch 8/40
51/51 [==============================] - 30s 598ms/step
ACC: 0.7044 AUC: 0.8859
Epoch 9/40
51/51 [==============================] - 30s 591ms/step
ACC: 0.7635 AUC: 0.8922
Epoch 10/40
51/51 [==============================] - 30s 597ms/step
ACC: 0.7783 AUC: 0.8992
Epoch 11/40
51/51 [==============================] - 30s 597ms/step
ACC: 0.7882 AUC: 0.8820


InternalError: Dst tensor is not initialized.
	 [[Node: training/SGD/gradients/replica_3/model_1/NASNet/adjust_bn_15/FusedBatchNorm_grad/FusedBatchNormGrad/_43501 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:3", send_device_incarnation=1, tensor_name="edge_18706...chNormGrad", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

In [ ]:
y_pred = model.predict_generator(gen_valid, verbose=1, workers=multiprocessing.cpu_count())
y_pred2 = 1 - y_pred[:,0]
y_pred3 = 1 - (np.argmax(y_pred, axis=-1) == 0)
pprint(roc_auc_score(y_valid, y_pred2))
pprint(accuracy_score(y_valid, y_pred3))
pprint(confusion_matrix(y_valid > 0.5, y_pred3))